# **Case #003: The Miami Marina Murder**
#### A body was found floating near the docks of Coral Bay Marina in the early hours of August 14, 1986. Your job detective is to find the murderer and bring them to justice. This case might require the use of JOINs, wildcard searches, and logical deduction. Get to work, detective.

## **Objectives**
    1. Find the murderer. ( Start by finding the crime scene and go from there )

##### First things first, what do we know?
    - location = Coral Bay Marina
    - date = Aug 14th, 1986
    - time = early hours of Aug 14th

##### **Step 1**: Review the crime scene report

In [ ]:
SELECT *
FROM crime_scene
WHERE location = 'Coral Bay Marina';

# dataset output
# id	date	location	description
# 43	19860814	Coral Bay Marina	The body of an unidentified man was found near the docks. Two people were seen nearby: one who lives on 300ish "Ocean Drive" and another whose first name ends with "ul" and his last name ends with "ez".

##### What do we know?
    - witnesses claim they saw 2 people near the crime scene
    - suspect #1 lives on 300ish Ocean Drive
    - suspect #2 first name ends with 'ul' and last name with 'ez'

##### **Step 2**: Locate Both Potential Suspects

In [ ]:
SELECT id
  , name
  , alias
  , occupation
  , address
FROM person
WHERE 
-- suspect #1 profile
address LIKE '%Ocean Drive%' OR
  
-- suspect #2 profile
(name LIKE  '%ul%' AND 
  name LIKE '%ez%');

# dataset output
# id	name	alias	address
# 1	Marco Romano	The Shadow	22 Ocean Drive
# 5	Michael Santos	Silent Mike	33 Ocean Drive
# 62	Jesse Brooks	The Judge	234 Ocean Drive
# 101	Carlos Mendez	Los Ojos	369 Ocean Drive
# 102	Raul Gutierrez	The Cobra	45 Sunset Ave
# 105	Victor Martinez	Slick Vic	33 Ocean Drive

8	Thomas Brown	The Fox	Dock Worker	234 Port Street	55	8	Sunset Palm Resort	19860813	8	8	2	Left suddenly at 3 AM	113	8	NULL	73	8	Alright! I did it. I was paid to make sure he never left the marina alive

##### **Step 3**: Investigating Both Witnesses' Interrogation Transcripts

In [ ]:
SELECT p.id
  , p.name
  , p.alias
  , i.transcript
FROM person p
JOIN interviews i ON i.person_id=p.id
WHERE p.id IN (101, 102);

# dataset output
# id	name	alias	transcript
# 101	Carlos Mendez	Los Ojos	I saw someone check into a hotel on August 13. The guy looked nervous.
# 102	Raul Gutierrez	The Cobra	I heard someone checked into a hotel with "Sunset" in the name.

##### **Checkpoint**: What do we know?
    - suspicious person checked into a hotel on Aug 13th with the hotel name having 'Sunset' in it

##### **Step 4**: Identify Suspicous Activity Noted at the Hotel

In [ ]:
SELECT hc.hotel_name
  , hc.check_in_date
  , hc.person_id
  , sr.suspicious_activity
FROM hotel_checkins hc
JOIN surveillance_records sr ON sr.hotel_checkin_id=hc.id
WHERE hc.hotel_name LIKE '%Sunset%' AND
  hc.check_in_date = '19860813';
  
# dataset output
# hotel_name	check_in_date	person_id	suspicious_activity
# Sunset Shore Hotel	19860813	17	NULL
# Sunset Shores Resort	19860813	32	Used the hotel gym
# Sunset View Inn	19860813	13	NULL
# Sunset Coast Inn	19860813	39	Spotted entering late at night
# Sunset Bay Hotel	19860813	27	Left suddenly at 3 AM
# . . . 

In [ ]:
SELECT p.id AS suspect_id
  , p.name AS suspect_name
  , p.occupation
  , hc.hotel_name
  , hc.check_in_date
  , sr.suspicious_activity
  , i.transcript
  , c.confession
FROM person p
JOIN hotel_checkins hc ON hc.person_id=p.id
JOIN surveillance_records sr ON sr.person_id=p.id
JOIN interviews i ON i.person_id=p.id
JOIN confessions c ON c.person_id=p.id
WHERE hc.hotel_name LIKE '%Sunset%' AND
  hc.check_in_date = '19860813' AND
  sr.suspicious_activity IS NOT NULL
ORDER BY p.id ASC;

# dataset output
# suspect_id	suspect_name	occupation	hotel_name	check_in_date	suspicious_activity	transcript	confession
# 7	Robert Smith	Mechanic	Sunset Marina Hotel	19860813	Seen arguing with an unknown person	NULL	I was just walking my dog that night.
# 8	Thomas Brown	Dock Worker	Sunset Palm Resort	19860813	Left suddenly at 3 AM	NULL	Alright! I did it. I was paid to make sure he never left the marina alive.
# 9	David Clark	Warehouse Manager	Sunset Beach Hotel	19860813	Requested wake-up call	There was an argument near slip 42 around sunset.	Who are you to question me? I'm an innocent man.
# . . . 

##### **Solution**: Thomas Brown